In [1]:
import warnings

import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel as C

warnings.filterwarnings("ignore")
from sklearn.gaussian_process.kernels import RBF
import pandas as pd

In [2]:
Xs = []
Ys = []
for i in range(1, 9):
    Xs.append(np.load(f"initial_data/function_{i}/initial_inputs.npy"))
    Y = np.load(f"initial_data/function_{i}/initial_outputs.npy")
    if i == 4:
        Y = np.abs(Y)
    Ys.append(Y)

for i in range(1, 9):
    Xs.append(np.load(f"initial_data2/function_{i}/initial_inputs.npy"))
    Y = np.load(f"initial_data2/function_{i}/initial_outputs.npy")
    if i == 4:
        Y = np.abs(Y)
    Ys.append(Y)

df = pd.read_csv('feedback_data/605_data.csv')
df = df.drop(columns=['Unnamed: 0', 'timestamp', 'student_id'], axis=1)

for index, col in enumerate(df.columns):
    if 'output' in col:
        y_array = np.array(df[col], dtype=float)
        Ys[index-8] = np.hstack((y_array, Ys[index-8]))
        continue
    series = []
    for i in range(len(df[col])):
        series.append(df[col][i].replace("[", "").replace("]", "").split())
    series_array = np.array(series, dtype=float)
    Xs[index] = np.vstack((series_array, Xs[index]))

In [3]:
def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01):
    X = np.atleast_2d(X)
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, 1)    # Make variance a column vector

    # Needed for noise-based models, otherwise I could have used np.max(Y_sample).
    mu_sample_opt = np.max(mu_sample)

    # Handle divide by zero with a warning
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

# Propose the next sampling point by optimizing the acquisition function
def propose_location(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=25):
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None

    def min_obj(X):
        return -acquisition(X.reshape(1, -1), X_sample, Y_sample, gpr)

    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B') # Limited memory Broyden–Fletcher–Goldfarb–Shanno algorithm
        if res.fun < min_val:
            min_val = res.fun
            min_x = res.x

    return min_x


In [4]:
print("EXPECTED IMPROVEMENT")
dims = [2, 2,  3, 4, 4, 5, 6, 8]
kernel = 1.0 * RBF(1.0)
for i, dim in enumerate(dims):
    bounds = np.array([[0, 1]] * dim)
    kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
    gp.fit(Xs[i], Ys[i])
    next_point = propose_location(expected_improvement, Xs[i], Ys[i], gp, bounds)
    next_query_string = "-".join([f"{x:.6f}" for x in next_point])
    print(f"The next query for the function {i+1} is {next_query_string}")

EXPECTED IMPROVEMENT
The next query for the function 1 is 0.452288-0.097389
The next query for the function 2 is 0.517129-0.902516
The next query for the function 3 is 0.000000-1.000000-0.000000
The next query for the function 4 is 0.328421-0.463792-0.486780-0.022579
The next query for the function 5 is 0.700718-0.307323-0.334003-0.810409
The next query for the function 6 is 0.322763-0.294062-0.040975-0.859567-0.776464
The next query for the function 7 is 0.137594-0.234672-0.411572-0.273354-0.311853-0.582084
The next query for the function 8 is 0.266038-0.381864-0.132140-0.636026-0.150824-0.781083-0.174686-0.404707
